In [1]:
#%%
from model import AutoEncoder
from utils import audio_process
import IPython.display as ipd
import tensorflow as tf
import numpy as np
import os

/home/halley/anaconda3/envs/tensorflow_2.1/lib/python3.6/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [7]:
#%%
'''
Usage:
  Interactive web page via jupyter
'''
ckpt_path = './ckpt'

auto_encoder = AutoEncoder()
dummy_voice = np.ones((1, 512, audio_process.n_mels, 1))
dummy_speaker = np.eye(2)[[0]]
auto_encoder(dummy_voice, dummy_speaker)
restored_ckpt = tf.train.Checkpoint(auto_encoder=auto_encoder)
latest_ckpt = tf.train.latest_checkpoint(ckpt_path)
restored_ckpt.restore(latest_ckpt).assert_existing_objects_matched()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [14]:
#%%
result_dir = 'result'
input_wav = '../dataset/wav48/p2/p2_002.wav'
target_speaker = 1
speaker_counts = 2

mag, mel = audio_process.wav2spectrogram(input_wav)
mel = np.transpose(mel.astype(np.float32))
mel = mel[np.newaxis, :, :, np.newaxis]
speaker = np.eye(speaker_counts)[target_speaker]
speaker = speaker[np.newaxis, :]

encoded, decoded = auto_encoder(mel, speaker)
converted_np = decoded.numpy()[0,:,:,0]

converted_np = np.transpose(converted_np.astype(np.float32))
converted_wav = audio_process.melspectrogram2wav(converted_np)

filename = input_wav.split('/')[-1].split('.')[0]
filename = filename + '_to_speaker%d.wav' % (target_speaker)
filepath = os.path.join(ckpt_path, result_dir, filename)
os.makedirs(os.path.dirname(filepath), exist_ok=True)
abs_filepath = audio_process.write_wav(converted_wav, filepath)
ipd.Audio(abs_filepath)